Notebook para la conversión de db sqlite to pg

In [1]:
import sqlite3
import pandas as pd
import psycopg2
from configparser import ConfigParser

Bloque 1: Sqlite Dump File or dataframe

In [3]:
database = "D:/BDD-Sqlite(V1)/app.db"
conn = sqlite3.connect(database)

In [4]:
df = pd.read_sql("select sql from sqlite_master", con=conn)
df = df.mask(df.eq(None)).dropna()

In [16]:
#for i in df.index:
#    print(df['sql'][i])
#print(df['sql'][0].index('a'))
a = df['sql'][12].replace('CHECK', '@')
b = a.replace(a[a.index('@'):len(a)-1], '')
temp = list(b)
temp[len(temp)-5] = ''
b = "".join(temp)
print(df['sql'][12])
print(b[:b.index(b[len(b)-5])] + '' + b[b.index(b[len(b)-5])+1:])
print(b)

CREATE TABLE warehouse (
	id INTEGER NOT NULL, 
	name VARCHAR(16), 
	location VARCHAR(16), 
	notes VARCHAR(255), 
	soft_delete BOOLEAN, 
	PRIMARY KEY (id), 
	CHECK (soft_delete IN (0, 1))
)
CREATE TABLE warehouse (
	id INTEGER NOT NULL, 
	name VARCHAR(16, 
	location VARCHAR(16), 
	notes VARCHAR(255), 
	soft_delete BOOLEAN, 
	PRIMARY KEY (id) 
	)
CREATE TABLE warehouse (
	id INTEGER NOT NULL, 
	name VARCHAR(16), 
	location VARCHAR(16), 
	notes VARCHAR(255), 
	soft_delete BOOLEAN, 
	PRIMARY KEY (id) 
	)


In [20]:
conn.close()

Bloque 2: Dump file or dataframe parse

In [25]:
#df.replace('DATETIME', 'TIMESTAMP')
parsed_df = []
for i in df.index:
#    if(aux.find('CHECK') == -1):
    aux = df['sql'][i]
    aux = aux.replace('DATETIME', 'TIMESTAMP')
    aux = aux.replace('user', 'user_')
    if(aux.find('CHECK') != -1):
        a = aux.replace('CHECK', '@')
        b = a.replace(a[a.index('@'):len(a)-1], '')
        temp = list(b)
        temp[len(temp)-5] = ''
        aux = "".join(temp)
    parsed_df.append(aux)


print(parsed_df)

['CREATE TABLE alembic_version (\n\tversion_num VARCHAR(32) NOT NULL, \n\tCONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)\n)', 'CREATE TABLE feedback (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(128), \n\tcreation_date TIMESTAMP, \n\tfinal_date TIMESTAMP, \n\tnotes TEXT, \n\tanswer TEXT, \n\tuser_name VARCHAR(16), \n\tPRIMARY KEY (id)\n)', 'CREATE TABLE item_category (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(32), \n\tPRIMARY KEY (id)\n)', 'CREATE UNIQUE INDEX ix_item_category_name ON item_category (name)', 'CREATE TABLE organization_category (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(32), \n\tPRIMARY KEY (id)\n)', 'CREATE UNIQUE INDEX ix_organization_category_name ON organization_category (name)', 'CREATE TABLE organization_client_category (\n\tid INTEGER NOT NULL, \n\tname VARCHAR(32), \n\tPRIMARY KEY (id)\n)', 'CREATE UNIQUE INDEX ix_organization_client_category_name ON organization_client_category (name)', 'CREATE TABLE user_ (\n\tid INTEGER NOT NULL, \n\tuser_name VARCHAR(1

Bloque 3: Creación de base de datos

Bloque 4: Dump file to pg

In [21]:
def config(archivo='pg.ini', seccion='postgresql'):
    # Crear el parser y leer el archivo
    parser = ConfigParser()
    parser.read(archivo)
 
    # Obtener la sección de conexión a la base de datos
    db = {}
    if parser.has_section(seccion):
        params = parser.items(seccion)
        for param in params:
            db[param[0]] = param[1]
        return db
    else:
        raise Exception('Secccion {0} no encontrada en el archivo {1}'.format(seccion, archivo))

In [22]:
def conectar():
    """ Conexión al servidor de bases de datos PostgreSQL """
    conexion = None
    try:
        # Lectura de los parámetros de conexion
        params = config()
 
        # Conexion al servidor de PostgreSQL
        print('Conectando a la base de datos PostgreSQL...')
        print(params)
        conexion = psycopg2.connect(**params)
        conexion.autocommit = True
 
        # creación del cursor
        cur = conexion.cursor()
        
        # Ejecución de una consulta con la version de PostgreSQL
        print('La version de PostgreSQL es la:')
        cur.execute('SELECT version()')
 
        # Ahora mostramos la version
        version = cur.fetchone()
        print(version)
       
       # Ejecutamos una consulta
       ## cur.execute( "SELECT nombre, apellidos FROM empleados" )

        # Recorremos los resultados y los mostramos
        ##for nombre, apellidos in cur.fetchall() :
        ##    print (nombre, apellidos)

        # Cierre de la comunicación con PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conexion is not None:
            """conexion.close()
            print('Conexión finalizada.')"""

In [23]:
conectar()

Conectando a la base de datos PostgreSQL...
{'host': '192.168.1.105', 'database': 'Prueba2', 'user': 'gdeluca', 'password': 'gdeluca', 'port': '5432'}
La version de PostgreSQL es la:
('PostgreSQL 14.2 (Ubuntu 14.2-1ubuntu1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)


In [27]:
# Creamos el cursor con el objeto conexion
params = config()
conexion = psycopg2.connect(**params)
conexion.autocommit = True
cur = conexion.cursor()
# Ejecutamos una consulta
for i in range(len(parsed_df)):
    cur.execute(parsed_df[i])

# Recorremos los resultados y los mostramos
"""for id, name in cur.fetchall() :
    print(id, name)"""

# Cerramos la conexión
conexion.commit()
conexion.close()